In [1]:
import pandas as pd

In [2]:
import pandas as pd

# Cargar el archivo Parquet
try:
    df_users_items = pd.read_parquet('fixed_users_items.parquet')
    print("Archivo Parquet cargado exitosamente.")
except Exception as e:
    print(f"Error al cargar el archivo Parquet: {e}")
    exit(1)

# Mostrar las primeras filas para verificar
print(df_users_items.head())
print(df_users_items.dtypes)

Archivo Parquet cargado exitosamente.
             user_id           steam_id  \
0            --000--  76561198084750368   
1            --ace--  76561198113770880   
2          --ionex--  76561198101468256   
3       -2SV-vuLB-Kg  76561197999208912   
4  -404PageNotFound-  76561198074247616   

                                         user_url  \
0            http://steamcommunity.com/id/--000--   
1            http://steamcommunity.com/id/--ace--   
2          http://steamcommunity.com/id/--ionex--   
3       http://steamcommunity.com/id/-2SV-vuLB-Kg   
4  http://steamcommunity.com/id/-404PageNotFound-   

                                               items  items_count  
0  [{'item_id': '4000', 'item_name': 'Garry's Mod...           58  
1  [{'item_id': '4000', 'item_name': 'Garry's Mod...           44  
2  [{'item_id': '10180', 'item_name': 'Call of Du...           23  
3  [{'item_id': '29800', 'item_name': 'Caster', '...           68  
4  [{'item_id': '240', 'item_name': 'Counter

{"user_id":"--000--","steam_id":"76561198084750368","user_url":"http://steamcommunity.com/id/--000--","items":[{"item_id":"4000","item_name":"Garry's Mod","playtime_2weeks":0,"playtime_forever":925},{"item_id":"1250","item_name":"Killing Floor","playtime_2weeks":0,"playtime_forever":2949},{"item_id":"35420","item_name":"Killing Floor Mod: Defence Alliance 2","playtime_2weeks":0,"playtime_forever":0}],"items_count":2}


Filtramos a usarios con items_count >= 200 para optimizar el dataset de entrenamiento


In [30]:
# Filtrar usuarios con 'items_count' >= 200
df_filtered_users = df_users_items[df_users_items['items_count'] >= 200].copy()
print(f"Número de usuarios después del filtrado: {df_filtered_users.shape[0]}")

Número de usuarios después del filtrado: 4812


Eliminamos los juegos con playtime_forever que no han jugado

In [31]:
# Eliminar juegos donde 'playtime_forever' == 0
def eliminar_juegos_no_jugados(items):
    return [item for item in items if item.get('playtime_forever', 0) > 0]

df_filtered_users['items'] = df_filtered_users['items'].apply(eliminar_juegos_no_jugados)




In [32]:
df_filtered_users

,user_id,steam_id,user_url,items,items_count
15,-Mad-,76561198031049280,http://steamcommunity.com/id/-Mad-,"[{'item_id': '10', 'item_name': 'Counter-Strik...",367
17,-PRoSlayeR-,76561198043569040,http://steamcommunity.com/id/-PRoSlayeR-,"[{'item_id': '8140', 'item_name': 'Tomb Raider...",529
19,-SEVEN-,76561198024539040,http://steamcommunity.com/id/-SEVEN-,"[{'item_id': '240', 'item_name': 'Counter-Stri...",394
46,005577,76561198089768368,http://steamcommunity.com/id/005577,"[{'item_id': '240', 'item_name': 'Counter-Stri...",231
48,0071945,76561198030091456,http://steamcommunity.com/id/0071945,"[{'item_id': '2100', 'item_name': 'Dark Messia...",373
...,...,...,...,...,...
87571,zomgieee,76561197974622848,http://steamcommunity.com/id/zomgieee,"[{'item_id': '10', 'item_name': 'Counter-Strik...",669
87584,zourock,76561197995258112,http://steamcommunity.com/id/zourock,"[{'item_id': '2100', 'item_name': 'Dark Messia...",290
87590,zsharoarkbr,76561198054666736,http://steamcommunity.com/id/zsharoarkbr,"[{'item_id': '3830', 'item_name': 'Psychonauts...",309
87591,zucchin1,76561198041637408,http://steamcommunity.com/id/zucchin1,"[{'item_id': '240', 'item_name': 'Counter-Stri...",234


In [33]:
# Opcional: Eliminar usuarios que ahora tienen 0 juegos después del filtrado
df_filtered_users = df_filtered_users[df_filtered_users['items'].map(len) > 0].copy()
print(f"Número de usuarios después de eliminar juegos no jugados: {df_filtered_users.shape[0]}")

Número de usuarios después de eliminar juegos no jugados: 4812


In [34]:
# Explode de la columna 'items' para tener una fila por juego jugado
df_exploded = df_filtered_users.explode('items')
print(f"Número de filas después del explode: {df_exploded.shape[0]}")

Número de filas después del explode: 958719


In [35]:
df_exploded

,user_id,steam_id,user_url,items,items_count
15,-Mad-,76561198031049280,http://steamcommunity.com/id/-Mad-,"{'item_id': '10', 'item_name': 'Counter-Strike...",367
15,-Mad-,76561198031049280,http://steamcommunity.com/id/-Mad-,"{'item_id': '20', 'item_name': 'Team Fortress ...",367
15,-Mad-,76561198031049280,http://steamcommunity.com/id/-Mad-,"{'item_id': '30', 'item_name': 'Day of Defeat'...",367
15,-Mad-,76561198031049280,http://steamcommunity.com/id/-Mad-,"{'item_id': '40', 'item_name': 'Deathmatch Cla...",367
15,-Mad-,76561198031049280,http://steamcommunity.com/id/-Mad-,"{'item_id': '50', 'item_name': 'Half-Life: Opp...",367
...,...,...,...,...,...
87603,zv_odd,76561197976793504,http://steamcommunity.com/id/zv_odd,"{'item_id': '420790', 'item_name': 'StarBreak'...",482
87603,zv_odd,76561197976793504,http://steamcommunity.com/id/zv_odd,"{'item_id': '205230', 'item_name': 'Hell Yeah!...",482
87603,zv_odd,76561197976793504,http://steamcommunity.com/id/zv_odd,"{'item_id': '205950', 'item_name': 'Jet Set Ra...",482
87603,zv_odd,76561197976793504,http://steamcommunity.com/id/zv_odd,"{'item_id': '374320', 'item_name': 'DARK SOULS...",482


In [36]:
# Seleccionar columnas relevantes y expandir la columna 'items'
df_exploded = df_exploded[['user_id', 'items']].copy()

# Expandir los diccionarios dentro de 'items' en columnas separadas
df_exploded = pd.concat([df_exploded.drop(['items'], axis=1), df_exploded['items'].apply(pd.Series)], axis=1)

# Renombrar columnas para mayor claridad
df_exploded.rename(columns={
    'item_id': 'game_id',
    'item_name': 'game_name',
    'playtime_2weeks': 'playtime_2weeks',
    'playtime_forever': 'playtime_forever'
}, inplace=True)

# Mostrar las primeras filas del DataFrame resultante
print(df_exploded.head())


   user_id game_id                  game_name  playtime_2weeks  \
15   -Mad-      10             Counter-Strike                0   
15   -Mad-      20      Team Fortress Classic                0   
15   -Mad-      30              Day of Defeat                0   
15   -Mad-      40         Deathmatch Classic                0   
15   -Mad-      50  Half-Life: Opposing Force                0   

    playtime_forever  
15                 8  
15                 3  
15                17  
15                20  
15                18  


In [37]:
# Crear una tabla de interacciones usuario-juego
df_user_game = df_exploded[['user_id', 'game_id']].copy()

# Mostrar las primeras filas de la tabla de interacciones
print(df_user_game.head())


   user_id game_id
15   -Mad-      10
15   -Mad-      20
15   -Mad-      30
15   -Mad-      40
15   -Mad-      50


In [38]:
# Crear una matriz usuario-juego
user_game_matrix = df_user_game.pivot_table(index='user_id', columns='game_id', aggfunc='size', fill_value=0)

# Convertir a binario (1 si ha jugado, 0 si no)
user_game_matrix_binary = user_game_matrix.applymap(lambda x: 1 if x > 0 else 0)

# Mostrar la matriz binaria
print(user_game_matrix_binary.head())


C:\Users\Art\AppData\Local\Temp\ipykernel_27544\4043963678.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  user_game_matrix_binary = user_game_matrix.applymap(lambda x: 1 if x > 0 else 0)


game_id      10  100  10000  1002  100400  100410  10080  10090  100970  \
user_id                                                                   
-Mad-         1    0      0     0       0       0      0      0       0   
-PRoSlayeR-   0    0      0     0       0       0      0      1       0   
-SEVEN-       1    0      0     0       0       0      0      0       0   
005577        0    0      0     0       0       0      0      0       0   
0071945       0    0      0     0       0       0      0      0       0   

game_id      100980  ...  9970  99700  9980  99810  99830  99890  9990  99900  \
user_id              ...                                                        
-Mad-             0  ...     0      0     0      0      0      0     0      0   
-PRoSlayeR-       0  ...     0      0     0      0      0      0     0      0   
-SEVEN-           0  ...     0      0     0      1      0      0     0      0   
005577            0  ...     0      0     0      0      0      0     

# Usamos similtud de coseno para usuario - item

In [39]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Calcular la similitud del coseno entre usuarios
similarity_matrix = cosine_similarity(user_game_matrix_binary)
similarity_df = pd.DataFrame(similarity_matrix, index=user_game_matrix_binary.index, columns=user_game_matrix_binary.index)

# Mostrar la matriz de similitud
print(similarity_df.head())


user_id         -Mad-  -PRoSlayeR-   -SEVEN-    005577   0071945  04061993  \
user_id                                                                      
-Mad-        1.000000     0.105670  0.257943  0.275835  0.233356  0.166945   
-PRoSlayeR-  0.105670     1.000000  0.143285  0.099388  0.058954  0.285395   
-SEVEN-      0.257943     0.143285  1.000000  0.174499  0.179148  0.225759   
005577       0.275835     0.099388  0.174499  1.000000  0.195832  0.147207   
0071945      0.233356     0.058954  0.179148  0.195832  1.000000  0.068109   

user_id      08alross    091263  095843067  1-800-Invisible  ...  zippysenpai  \
user_id                                                      ...                
-Mad-        0.234275  0.220316   0.220802         0.260824  ...     0.205938   
-PRoSlayeR-  0.077604  0.167319   0.054652         0.042654  ...     0.040414   
-SEVEN-      0.167693  0.212682   0.106288         0.193561  ...     0.181940   
005577       0.369199  0.210377   0.227321      

In [40]:
def recomendacion_usuario(user_id, df_user_game, similarity_df, df_exploded, num_recommendations=5):
    if user_id not in similarity_df.index:
        return f"Usuario {user_id} no encontrado."
    
    # Obtener las similitudes del usuario con todos los demás usuarios
    similar_users = similarity_df[user_id].sort_values(ascending=False)
    
    # Excluir el propio usuario
    similar_users = similar_users.drop(labels=[user_id])
    
    # Seleccionar los usuarios más similares (puedes ajustar el número según necesites)
    top_similar_users = similar_users.head(5).index
    
    # Obtener los juegos que han jugado los usuarios similares
    recommended_games = df_user_game[df_user_game['user_id'].isin(top_similar_users)]['game_id'].tolist()
    
    # Obtener los juegos que el usuario ya ha jugado
    user_games = df_user_game[df_user_game['user_id'] == user_id]['game_id'].tolist()
    
    # Filtrar los juegos que el usuario no ha jugado
    recommended_games = [game for game in recommended_games if game not in user_games]
    
    # Contar la frecuencia de los juegos recomendados
    recommended_games_series = pd.Series(recommended_games).value_counts()
    
    # Seleccionar los juegos más populares entre los usuarios similares
    recommended_games = recommended_games_series.head(num_recommendations).index.tolist()
    
    # Obtener los nombres de los juegos a partir de sus IDs
    recommended_game_names = df_exploded[df_exploded['game_id'].isin(recommended_games)]['game_name'].unique().tolist()
    
    return recommended_game_names


In [41]:
user_id_example = '-PRoSlayeR-'
recommended_games = recomendacion_usuario(user_id_example, df_user_game, similarity_df, df_exploded, num_recommendations=5)

print(f"Juegos recomendados para el usuario {user_id_example}:")
print(recommended_games)


Juegos recomendados para el usuario -PRoSlayeR-:
['Attrition: Nuclear Domination', 'Forsaken Uprising', 'Endorlight', 'Sumo Revise', 'Medieval Mercs']


## como el tamano de dataset es grande y trataremos de generar la mayor cantidad de ingresos posibles vamos a usar solo los juegos donde el price sea mayor a 20 dolares y tenga fecha de release valida o existente. adicionalmente si no tiene publisher no los utilizaremos ya que eso define la calidad de juego.

In [55]:
import pandas as pd

# Cargar el archivo Parquet de juegos
try:
    df_steam_games = pd.read_parquet('fixed_steam_games.parquet')
    print("Archivo 'fixed_steam_games.parquet' cargado exitosamente.")
except Exception as e:
    print(f"Error al cargar el archivo Parquet: {e}")
    exit(1)


Archivo 'fixed_steam_games.parquet' cargado exitosamente.


In [56]:
# Filtrar juegos con precio > 20
df_filtered_games = df_steam_games[df_steam_games['price'] > 20].copy()

# Filtrar juegos con fecha de lanzamiento válida
df_filtered_games = df_filtered_games[df_filtered_games['release_date'].notnull()].copy()

# Filtrar juegos con publisher definido
df_filtered_games = df_filtered_games[df_filtered_games['publisher'].notnull() & (df_filtered_games['publisher'] != '')].copy()

# Resetear el índice
df_filtered_games.reset_index(drop=True, inplace=True)

print(f"Número de juegos después del filtrado: {df_filtered_games.shape[0]}")


Número de juegos después del filtrado: 1474


In [57]:
# Mostrar las primeras filas del DataFrame filtrado
print(df_filtered_games.head())

# Verificar tipos de datos
print(df_filtered_games.dtypes)


                 publisher                                     genres  \
0   Poppermost Productions  [Free to Play, Indie, Simulation, Sports]   
1   Poppermost Productions  [Free to Play, Indie, Simulation, Sports]   
2                   Mode 7                          [Indie, Strategy]   
3  Activision, Aspyr (Mac)                                   [Action]   
4      Paradox Interactive                [RPG, Simulation, Strategy]   

                           app_name                             title  \
0     SNOW - All Access Legend Pass     SNOW - All Access Legend Pass   
1              SNOW - Lifetime Pack              SNOW - Lifetime Pack   
2                    Frozen Synapse                    Frozen Synapse   
3  Call of Duty®: Modern Warfare® 3  Call of Duty®: Modern Warfare® 3   
4                 Crusader Kings II                 Crusader Kings II   

                                                 url release_date  \
0  http://store.steampowered.com/app/774278/SNOW_... 

In [58]:
# Cargar el archivo Parquet de usuarios
try:
    df_users_items = pd.read_parquet('fixed_users_items.parquet')
    print("Archivo 'fixed_users_items.parquet' cargado exitosamente.")
except Exception as e:
    print(f"Error al cargar el archivo Parquet: {e}")
    exit(1)


Archivo 'fixed_users_items.parquet' cargado exitosamente.


In [46]:
# Filtrar usuarios con 'items_count' >= 200
df_filtered_users = df_users_items[df_users_items['items_count'] >= 200].copy()
print(f"Número de usuarios después del filtrado: {df_filtered_users.shape[0]}")


Número de usuarios después del filtrado: 4812


In [59]:
def eliminar_juegos_no_jugados_y_filtrar(items, valid_game_ids):
    # Filtrar juegos con 'playtime_forever' > 0 y que estén en 'valid_game_ids'
    return [item for item in items if item.get('playtime_forever', 0) > 0 and int(item.get('item_id')) in valid_game_ids]

# Obtener el conjunto de IDs de juegos válidos después del filtrado
valid_game_ids = set(df_filtered_games['id'].astype(int))

# Aplicar la función al DataFrame de usuarios
df_filtered_users['items'] = df_filtered_users['items'].apply(lambda items: eliminar_juegos_no_jugados_y_filtrar(items, valid_game_ids))

# Eliminar usuarios que ahora tienen 0 juegos después del filtrado
df_filtered_users = df_filtered_users[df_filtered_users['items'].map(len) > 0].copy()
print(f"Número de usuarios después de filtrar juegos no válidos: {df_filtered_users.shape[0]}")


Número de usuarios después de filtrar juegos no válidos: 4764


In [60]:
# Explode de la columna 'items'
df_exploded = df_filtered_users.explode('items')
print(f"Número de filas después del explode: {df_exploded.shape[0]}")


Número de filas después del explode: 78470


In [61]:
# Expandir los diccionarios dentro de 'items' en columnas separadas
df_exploded = pd.concat([df_exploded.drop(['items'], axis=1), df_exploded['items'].apply(pd.Series)], axis=1)

# Renombrar columnas
df_exploded.rename(columns={
    'item_id': 'game_id',
    'item_name': 'game_name',
    'playtime_2weeks': 'playtime_2weeks',
    'playtime_forever': 'playtime_forever'
}, inplace=True)


In [62]:
# Crear la tabla de interacciones
df_user_game = df_exploded[['user_id', 'game_id']].copy()


In [63]:
# Crear la matriz usuario-juego
user_game_matrix = df_user_game.pivot_table(index='user_id', columns='game_id', aggfunc='size', fill_value=0)

# Convertir a binario
user_game_matrix_binary = user_game_matrix.applymap(lambda x: 1 if x > 0 else 0)


C:\Users\Art\AppData\Local\Temp\ipykernel_27544\2255944565.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  user_game_matrix_binary = user_game_matrix.applymap(lambda x: 1 if x > 0 else 0)


In [64]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcular la similitud del coseno
similarity_matrix = cosine_similarity(user_game_matrix_binary)

# Crear un DataFrame de similitud
similarity_df = pd.DataFrame(similarity_matrix, index=user_game_matrix_binary.index, columns=user_game_matrix_binary.index)


In [65]:
# Asegurarnos de que 'genres' sea una lista
df_filtered_games['genres'] = df_filtered_games['genres'].apply(lambda x: x if isinstance(x, list) else [])



In [66]:
# Combinar características relevantes
df_filtered_games['features'] = df_filtered_games['genres'].apply(lambda x: ' '.join(x) if isinstance(x, list) else '') + ' ' + \
                                df_filtered_games['developer'].fillna('') + ' ' + \
                                df_filtered_games['app_name'].fillna('')


In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Vectorizar las características
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_filtered_games['features'])

# Calcular similitud del coseno entre juegos
cosine_sim_games = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Crear un índice para mapear nombres de juegos a índices
indices_games = pd.Series(df_filtered_games.index, index=df_filtered_games['app_name']).drop_duplicates()


In [68]:
def recomendacion_contenido(game_name, cosine_sim=cosine_sim_games):
    # Verificar si el juego existe en el índice
    if game_name not in indices_games:
        return []
    
    # Obtener el índice del juego
    idx = indices_games[game_name]
    
    # Obtener las puntuaciones de similitud para todos los juegos con respecto al juego dado
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Ordenar los juegos por puntuación de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtener los índices de los juegos más similares
    sim_scores = sim_scores[1:6]  # Ignorar el primero ya que es el mismo juego
    
    # Obtener los índices de los juegos recomendados
    game_indices = [i[0] for i in sim_scores]
    
    # Devolver los nombres de los juegos recomendados
    return df_filtered_games['app_name'].iloc[game_indices].tolist()


In [69]:
# Convertir 'developer' y 'app_name' a cadenas de texto y manejar valores no esperados
df_filtered_games['developer'] = df_filtered_games['developer'].astype(str).fillna('')
df_filtered_games['app_name'] = df_filtered_games['app_name'].astype(str).fillna('')


In [70]:
def recomendacion_contenido(game_name, cosine_sim=cosine_sim_games):
    # Verificar si el juego existe en el índice
    if game_name not in indices_games:
        return []
    
    # Obtener el índice del juego
    idx = indices_games[game_name]
    
    # Obtener las puntuaciones de similitud para todos los juegos con respecto al juego dado
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Ordenar los juegos por puntuación de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtener los índices de los juegos más similares
    sim_scores = sim_scores[1:6]  # Ignorar el primero ya que es el mismo juego
    
    # Obtener los índices de los juegos recomendados
    game_indices = [i[0] for i in sim_scores]
    
    # Devolver los nombres de los juegos recomendados
    return df_filtered_games['app_name'].iloc[game_indices].tolist()



In [71]:
def recomendacion_usuario(user_id, df_user_game, similarity_df, df_exploded, df_filtered_games, num_recommendations=5):
    # Verificar si el usuario está en el conjunto filtrado
    if user_id in similarity_df.index:
        # Recomendación basada en filtrado colaborativo
        similar_users = similarity_df[user_id].sort_values(ascending=False)
        similar_users = similar_users.drop(labels=[user_id])
        top_similar_users = similar_users.head(5).index
        recommended_games = df_user_game[df_user_game['user_id'].isin(top_similar_users)]['game_id'].tolist()
        user_games = df_user_game[df_user_game['user_id'] == user_id]['game_id'].tolist()
        recommended_games = [game for game in recommended_games if game not in user_games]
        recommended_games_series = pd.Series(recommended_games).value_counts()
        recommended_games = recommended_games_series.head(num_recommendations).index.tolist()
        # Obtener los nombres de los juegos a partir de sus IDs
        recommended_game_names = df_filtered_games[df_filtered_games['id'].astype(str).isin(recommended_games)]['app_name'].unique().tolist()
        return recommended_game_names
    else:
        # Recomendación basada en contenido
        user_games = df_user_game[df_user_game['user_id'] == user_id]['game_id'].tolist()
        if not user_games:
            return "No hay juegos jugados por este usuario para generar recomendaciones."
        user_game_names = df_filtered_games[df_filtered_games['id'].astype(str).isin(user_games)]['app_name'].tolist()
        recommended_game_names = []
        for game in user_game_names:
            recs = recomendacion_contenido(game)
            recommended_game_names.extend(recs)
        recommended_game_names = [game for game in recommended_game_names if game not in user_game_names]
        recommended_game_names = list(pd.Series(recommended_game_names).drop_duplicates().head(num_recommendations))
        return recommended_game_names


## Preparamos la exportacion de archivos a el Endpoint en FastAPI

In [72]:
# Guardar DataFrames en formato parquet
df_user_game.to_parquet('df_user_game.parquet')
similarity_df.to_parquet('similarity_df.parquet')
df_filtered_games.to_parquet('df_filtered_games.parquet')
df_exploded.to_parquet('df_exploded.parquet')

# Opcionalmente, guardar modelos con joblib
import joblib
joblib.dump(cosine_sim_games, 'cosine_sim_games.pkl')
joblib.dump(indices_games, 'indices_games.pkl')

['indices_games.pkl']

## FAST API ENDPOINT

### def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. 

In [74]:
# Asegurarnos de que df_filtered_games está cargado
try:
    df_filtered_games
except NameError:
    # Si no está cargado, cargar el DataFrame
    df_filtered_games = pd.read_parquet('df_filtered_games.parquet')


In [75]:
# Crear la columna 'free' si no existe
if 'free' not in df_filtered_games.columns:
    df_filtered_games['free'] = df_filtered_games['price'].apply(lambda x: True if x == 0 else False)


In [76]:
df_filtered_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,features,free
0,Poppermost Productions,[],SNOW - All Access Legend Pass,SNOW - All Access Legend Pass,http://store.steampowered.com/app/774278/SNOW_...,2018-01-04,"[Free to Play, Indie, Simulation, Sports]",http://steamcommunity.com/app/774278/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",29.99,False,774278,Poppermost Productions,Poppermost Productions SNOW - All Access Lege...,False
1,Poppermost Productions,[],SNOW - Lifetime Pack,SNOW - Lifetime Pack,http://store.steampowered.com/app/774279/SNOW_...,2018-01-03,"[Free to Play, Indie, Simulation, Sports]",http://steamcommunity.com/app/774279/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",59.99,False,774279,Poppermost Productions,Poppermost Productions SNOW - Lifetime Pack,False
2,Mode 7,[],Frozen Synapse,Frozen Synapse,http://store.steampowered.com/app/98200/Frozen...,2011-05-26,"[Strategy, Turn-Based, Tactical, Indie, Asynch...",http://steamcommunity.com/app/98200/reviews/?b...,"[Single-player, Multi-player, Steam Achievemen...",24.99,False,98200,Mode 7,Mode 7 Frozen Synapse,False
3,"Activision, Aspyr (Mac)",[],Call of Duty®: Modern Warfare® 3,Call of Duty®: Modern Warfare® 3,http://store.steampowered.com/app/115300/Call_...,2011-11-08,"[Action, FPS, Multiplayer, Shooter, First-Pers...",http://steamcommunity.com/app/115300/reviews/?...,"[Single-player, Multi-player, Co-op, Steam Ach...",39.99,False,115300,"Infinity Ward,Sledgehammer Games,Aspyr (Mac)","Infinity Ward,Sledgehammer Games,Aspyr (Mac) ...",False
4,Paradox Interactive,[],Crusader Kings II,Crusader Kings II,http://store.steampowered.com/app/203770/Crusa...,2012-02-14,"[Grand Strategy, Strategy, Medieval, Historica...",http://steamcommunity.com/app/203770/reviews/?...,"[Single-player, Multi-player, Steam Trading Ca...",39.99,False,203770,Paradox Development Studio,Paradox Development Studio Crusader Kings II,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1469,Electronic Arts,[],Mass Effect 2 Digital Deluxe Edition,Mass Effect 2 Digital Deluxe Edition,http://store.steampowered.com/app/901242/Mass_...,2010-01-27,"[RPG, Sci-fi, Story Rich, Action, Third-Person...",http://steamcommunity.com/app/901242/reviews/?...,[Single-player],29.99,False,901242,BioWare,BioWare Mass Effect 2 Digital Deluxe Edition,False
1470,Firedog Creative Company Limited,[],愛神餐館2,愛神餐館2,http://store.steampowered.com/app/769680/2/,2017-12-29,"[RPG, Simulation, Anime]",http://steamcommunity.com/app/769680/reviews/?...,[Single-player],34.99,False,769680,Firedog Creative Company Limited,Firedog Creative Company Limited 愛神餐館2,False
1471,CarX Technologies,[],CarX Drift Racing Online - Deluxe,CarX Drift Racing Online - Deluxe,http://store.steampowered.com/app/776950/CarX_...,2017-12-28,"[Massively Multiplayer, Simulation, Racing, Sp...",http://steamcommunity.com/app/776950/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",29.99,False,776950,CarX Technologies,CarX Technologies CarX Drift Racing Online - ...,False
1472,DreamInCode B.V.,[],Mesozoica,Mesozoica,http://store.steampowered.com/app/719590/Mesoz...,2018-01-02,"[Early Access, Simulation, Strategy, Casual, I...",http://steamcommunity.com/app/719590/reviews/?...,[Single-player],29.99,True,719590,"DreamInCode B.V.,Squadron Interactive","DreamInCode B.V.,Squadron Interactive Mesozoica",False


In [77]:
# Convertir 'release_date' a tipo datetime si no lo está
df_filtered_games['release_date'] = pd.to_datetime(df_filtered_games['release_date'], errors='coerce')

# Crear una columna 'year' con el año de lanzamiento
df_filtered_games['year'] = df_filtered_games['release_date'].dt.year


In [78]:
def developer(desarrollador: str):
    # Filtrar los juegos del desarrollador dado
    df_dev = df_filtered_games[df_filtered_games['developer'].str.contains(desarrollador, case=False, na=False)].copy()
    
    if df_dev.empty:
        return None  # O puedes devolver un mensaje indicando que no se encontraron resultados
    
    # Agrupar por año y calcular la cantidad de items y el porcentaje de contenido gratuito
    df_summary = df_dev.groupby('year').agg(
        cantidad_items=('id', 'count'),
        contenido_free=('free', lambda x: round((x.sum() / x.count()) * 100, 2))
    ).reset_index()
    
    # Ordenar por año descendente
    df_summary = df_summary.sort_values(by='year', ascending=False)
    
    # Convertir a lista de diccionarios
    result = df_summary.to_dict(orient='records')
    
    return result
